In [1]:
import json
import pandas as pd
import folium
from geopy.distance import geodesic

### Para obtener las distancias se ha realizado una consulta a la api de [OpenStreetMap](https://overpass-turbo.eu/)

<div style="text-align: center;">
  <a href="https://www.openstreetmap.org/#map=7/53.462/-8.240">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Openstreetmap_logo.svg/1024px-Openstreetmap_logo.svg.png" alt="OpenStreetMap" width="200" height="200">
  </a>
  <a href="https://overpass-turbo.eu/">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Overpass-turbo.svg/1024px-Overpass-turbo.svg.png" alt="Overpass Turbo" width="200" height="200">
  </a>
</div>


In [6]:
house = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/result_barcelona_districs_zip_final.csv', sep = ";")

In [7]:
supermercado = pd.read_csv('/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_coordenadas_servicios/supermercados_bcn.csv', sep = ";")

In [32]:
supermercado.columns

Index(['Parque', 'longitude', 'latitude'], dtype='object')

In [8]:
nom = supermercado.columns[0]
nom_dist = nom + '_distancia'
nom_dist

'Supermercado_distancia'

In [34]:
house['longitude'].isna().sum()

np.int64(0)

In [9]:
supermercado = supermercado[supermercado['longitude'] >= 0]
supermercado = supermercado[supermercado['latitude'] >= 0]

In [ ]:
def obtener_estacion_mas_cercana(house_row, supermercado_df):
    if 'latitude' not in house_row or 'longitude' not in house_row:
        raise ValueError("La fila de la casa no tiene columnas 'latitude' o 'longitude'")

    house_coords = (house_row['latitude'], house_row['longitude'])

    supermercado_df[nom_dist] = supermercado_df.apply(lambda supermercado_row: 
                                              geodesic(house_coords, (supermercado_row['latitude'], supermercado_row['longitude'])).meters, 
                                              axis=1)

    closest_station_row = supermercado_df.loc[supermercado_df[nom_dist].idxmin()]

    estacion_cercana = closest_station_row[nom]
    distancia_minima = closest_station_row[nom_dist]

    return estacion_cercana, distancia_minima

supermercado[['longitude', 'latitude']] = supermercado[['longitude', 'latitude']].apply(pd.to_numeric, errors='coerce')

house[['latitude', 'longitude']] = house[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

house[[nom, nom_dist]] = house.apply(
    lambda row: pd.Series(obtener_estacion_mas_cercana(row, supermercado)), axis=1
)

Index(['Supermercado', 'latitude', 'longitude', 'Supermercado_distancia'], dtype='object')
      propertyCode                                          thumbnail  \
0         99117414  https://img4.idealista.com/blur/WEB_LISTING-M/...   
1        101300670  https://img4.idealista.com/blur/WEB_LISTING-M/...   
2        104772587  https://img4.idealista.com/blur/WEB_LISTING-M/...   
3        100770574  https://img4.idealista.com/blur/WEB_LISTING-M/...   
4        105932101  https://img4.idealista.com/blur/WEB_LISTING-M/...   
...            ...                                                ...   
8913     105882581  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8914      81411842  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8915     103461663  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8916      98964029  https://img4.idealista.com/blur/WEB_LISTING-M/...   
8917     105093241  https://img4.idealista.com/blur/WEB_LISTING-M/...   

     externalReference  numPhoto

In [20]:
house[nom].value_counts()

Teatro 
Auditori Barradas       670
Teatre Joventut         643
Robert Brillas          594
Blas Infante            405
Auditori                386
                       ... 
Sala 3 Tete Montoliu      2
Sala Ars                  1
Barts                     1
Cal Ninyo                 1
Auditorium municipal      1
Name: count, Length: 84, dtype: int64

In [13]:
ultima = house.columns[-1]
penultima = house.columns[-2]

In [14]:
house = house[['propertyCode', penultima, ultima]]
house

,propertyCode,Supermercado,Supermercado_distancia
0,99117414,Coaliment,100.775853
1,101300670,Suma,336.944901
2,104772587,Primaprix,153.472939
3,100770574,Covirán,61.496949
4,105932101,Charter,28.021051
...,...,...,...
8913,105882581,Condis,7.235334
8914,81411842,Supermercat,202.114149
8915,103461663,Supermercat,627.468401
8916,98964029,Supermercat,1330.701309


In [15]:
house.to_csv("/TFM Mercado Inmobiliario - Kschool/Metodología de Modelado/9. Código y datasets/Barcelona/Datasets/Distancia_servicios/Dataset_distancias/" + nom_dist + ".csv", index=False, sep=";")
